# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [2]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 7.94MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [3]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
 !pip uninstall LLMChain

In [4]:
!pip install langchain

In [18]:
!pip install langchain-groq

In [5]:
from langchain.chains import LLMChain

# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [6]:
import pandas as pd

df = pd.read_csv("Dataset.csv")
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [7]:
df.shape

(104, 2)

In [8]:
df.duplicated().sum()

0

In [9]:
df.isna().sum()

,0
المخالفة,0
الغرامة,0


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [10]:
import os

directory = 'data/markdown_files'
os.makedirs(directory, exist_ok=True)

In [11]:
df.iterrows?

In [12]:
markdown_list = []

for index, row in df.iterrows():
    violation = row['المخالفة']
    fine = row['الغرامة']
    markdown = f"المخالفة: {violation} - الغرامة: {fine}"
    markdown_list.append(markdown)

    with open(f'{directory}/{index}.md', 'w', encoding='utf-8') as file:
        file.write(markdown)

In [13]:
markdown_list[10:15]

['المخالفة: مخالفة قواعد استعمال أنوار التلاقي. - الغرامة: الغرامة المالية 150 - 300 ريال',
 'المخالفة: عدم حمل رخصة القيادة أو رخصة السير أثناء القيادة. - الغرامة: الغرامة المالية 150 - 300 ريال',
 'المخالفة: إساءة استعمال منبه المركبة. - الغرامة: الغرامة المالية 150 - 300 ريال',
 'المخالفة: عدم تقديم المركبة للفحص الفني الدوري. - الغرامة: الغرامة المالية 150 - 300 ريال',
 'المخالفة: مخالفة تنظيمات السير على الطرق. - الغرامة: الغرامة المالية 150 - 300 ريال']

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.create_documents(markdown_list)

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [15]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

# embeddings_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings_model = SentenceTransformerEmbeddings(model_name="Omartificial-Intelligence-Space/Arabic-Triplet-Matryoshka-V2")
db = Chroma.from_documents(chunks, embeddings_model, persist_directory="./chroma_db")

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
<ipython-input-15-9f6523e402e2>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings_model = SentenceTransformerEmbeddings(model_name="Omartificial-Intelligence-Space/Arabic-Triplet-Matryoshka-V2")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqd

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/761k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [43]:
PRESIST_DIRECTORY = '/content/chroma_db'
embeddings_model = SentenceTransformerEmbeddings(model_name="Omartificial-Intelligence-Space/Arabic-Triplet-Matryoshka-V2")
persist_directory = "./chroma_db"
db = Chroma(persist_directory=persist_directory, embedding_function=embeddings_model)

In [60]:
def query_chroma_db(query, db, top_k=5):
    docs = db.similarity_search(query)
    results = [doc.page_content for doc in docs]
    return results

# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [96]:
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE="""
السؤال: {query}
الجواب: {context}
"""

prompt_template = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "query"]
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [97]:
# from langchain.llms import Groq
from langchain_groq import ChatGroq

groq_api_key = "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"
llm = ChatGroq(temperature=0.7,
               groq_api_key=groq_api_key,
               model_name="llama3-8b-8192") # model="gpt-4"

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [98]:
rag_chain = LLMChain(llm=llm,
                 prompt=prompt_template,
                 verbose=1)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [99]:
def query_rag(query: str):
    similarity_search_results = db.similarity_search_with_score(query, k=4)
    context_text = "\n\n".join([doc.page_content for doc, _score in similarity_search_results])

    rag_response = rag_chain.invoke({"context": context_text, "query": query})

    return rag_response

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [102]:
# Test a query
user_query = "ماهي الغرامة على تجاوز حد السرعة؟"
response = query_rag(user_query)
response

Prompt after formatting:

السؤال: ماهي الغرامة على تجاوز حد السرعة؟
الجواب: المخالفة: المراوغة بسرعة بين المركبات على الطرق العامة. - الغرامة: الغرامة المالية 3000 - 6000 ريال

المخالفة: عدم الالتزام بحدود المسارات المحددة على الطريق. - الغرامة: الغرامة المالية 300 - 500 ريال

المخالفة: عدم ربط حزام الأمان. - الغرامة: الغرامة المالية 150 - 300 ريال

المخالفة: تجاوز إشارة المرور الضوئية أثناء الضوء الأحمر. - الغرامة: الغرامة المالية 3000 - 6000 ريال


> Finished chain.


{'context': 'المخالفة: المراوغة بسرعة بين المركبات على الطرق العامة. - الغرامة: الغرامة المالية 3000 - 6000 ريال\n\nالمخالفة: عدم الالتزام بحدود المسارات المحددة على الطريق. - الغرامة: الغرامة المالية 300 - 500 ريال\n\nالمخالفة: عدم ربط حزام الأمان. - الغرامة: الغرامة المالية 150 - 300 ريال\n\nالمخالفة: تجاوز إشارة المرور الضوئية أثناء الضوء الأحمر. - الغرامة: الغرامة المالية 3000 - 6000 ريال',
 'query': 'ماهي الغرامة على تجاوز حد السرعة؟',
 'text': 'According to the given answers, the fine for exceeding the speed limit is:\n\n* 3000 - 6000 Saudi Riyals (SR)'}

In [103]:
# Test a query
user_query = "ماهي الغرامة على القيادة بدون رخصة"
response = query_rag(user_query)
response

Prompt after formatting:

السؤال: ماهي الغرامة على القيادة بدون رخصة
الجواب: المخالفة: عدم حمل رخصة القيادة أو رخصة السير أثناء القيادة. - الغرامة: الغرامة المالية 150 - 300 ريال

المخالفة: قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها. - الغرامة: الغرامة المالية 1000 - 2000 ريال

المخالفة: القيادة برخصة قيادة صلاحيتها منتهية. - الغرامة: الغرامة المالية 300 - 500 ريال

المخالفة: قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها. - الغرامة: الغرامة المالية 100 - 150 ريال


> Finished chain.


{'context': 'المخالفة: عدم حمل رخصة القيادة أو رخصة السير أثناء القيادة. - الغرامة: الغرامة المالية 150 - 300 ريال\n\nالمخالفة: قيادة المركبة برخصة قيادة لا تتناسب مع حجم المركبة ونوع استخدامها. - الغرامة: الغرامة المالية 1000 - 2000 ريال\n\nالمخالفة: القيادة برخصة قيادة صلاحيتها منتهية. - الغرامة: الغرامة المالية 300 - 500 ريال\n\nالمخالفة: قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها. - الغرامة: الغرامة المالية 100 - 150 ريال',
 'query': 'ماهي الغرامة على القيادة بدون رخصة',
 'text': "It seems like you're asking about the fines for driving without a license in Saudi Arabia. Here are the answers:\n\n1. What is the fine for driving without a license?\nThe offense: Not carrying a driver's license or driving permit while driving.\nThe fine: SR 150-300.\n\n2. What is the fine for driving a vehicle with a license that doesn't match the vehicle's size and use?\nThe offense: Driving a vehicle with a driver's license that doesn't match the vehicle's size and use.\nThe fine: SR 1000-200